# New York CitiBike Data Science Challenge

Der Fahrradverleih CitiBike vermietet in New York über 12.000 Fahrräder an 750 
Verleihstationen. Somit ist CitiBike eine echte Alternative zu den herkömmlichen 
Transportmitteln, wie z.B. U-bahn oder Taxi. 

Nehme an, es wäre das Jahr 2018. Mit einem gültigen 24 Stunden (3 Tage) Pass bzw. einer 
jährlichen Mitgliedschaft können Kunden ein Fahrrad an einer Verleihstation abholen und an 
einer beliebigen Station wieder abgeben. CitiBike stellt die durch den Verleih gesammelten 
Daten der Öffentlichkeit zur Verfügung. Deine Aufgabe als Data Scientist ist es, CitiBike dabei zu
helfen diese Daten wertstiftend zu nutzen. 

In einem ersten Pilotprojekt sollst du hierfür die Daten analysieren und ein Modell bauen, mit 
welchem zwei Klassen von Nutzern identifiziert werden können; dies sind (i) customers (24 
Stunden/ 3 Tage Pass) und (ii) subscribers (jährliche Mitgliedschaft).

Eine Orientierungshilfe bei 
dieser Aufgabenstellung bieten dir die folgenen Arbeitsschritte:
 
1. Lade diese Daten von Citibike für das Jahr 2018 herunter. Mach dich mit dem Inhalt des 
Datensatzes vertraut und bereite ihn für weitere Analysen auf. 
2. Visualisiere die Daten; sei kreativ und überlege dir geeignete Darstellungsformen für 
deine Entdeckungen. Nutze die Visualisierungen auch in deiner Ergebnispräsentation um 
deine Argumente zu unterstützen.
3. Überlege dir geeignete Features (Merkmale) als Input für dein customer-subscriber-
Modell. Konstruiere gegebenenfalls neue Features um dein Modell zu verbessern. 
4. Verteste verschiedene Modelle bzw. Methoden zur Klassifikation der Kundentypen 
subscribers und customers. Evaluiere die Performance der unterschiedlichen Modelle 
und begründe eine Modellauswahl.
5. Skizziere mögliche Einsatzgebiete oder UseCases deiner Modelle
6. Skizziere für CitiBike Kooperationsmöglichkeiten mit einer Versicherung (und/oder 
umgekehrt). 
 


## Imports und Co

In [ ]:
# Generelles
import os

# Data science
import numpy as np
import pandas as pd

# Machine learning
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import shap

# Plotting
import matplotlib.pyplot as plt

# Hilfen
from utils import crow_distance
from utils import forward_feature_selection

# Display
pd.set_option('display.max_columns', 500)
%matplotlib inline 

# Random seed
random_seed = 1
np.random.seed(random_seed)

# Konstanten
data_drop = 0.99
max_trip_duration = 12 * 3600

## Datenvorbereitung

In [ ]:
# Rohdaten einlesen, wir beschränken uns auf 1% der Daten um meine virtuelle Maschine zu schonen
data_directory = 'data_NY/'
files = sorted(os.listdir(data_directory))
for index, file in enumerate(files):
    if index == 0:
        data = pd.read_csv(data_directory+file)
        data.drop(np.random.choice(data.index, int(len(data)*data_drop), replace=False), inplace=True)
    else:
        data_tmp = pd.read_csv(data_directory+file)
        data_tmp.drop(np.random.choice(data_tmp.index, int(len(data_tmp)*data_drop), replace=False), inplace=True)
        data = pd.concat([data, data_tmp], ignore_index=True)

# Datentyp verbessern
for column in data.columns:
    data.rename(columns={column: column.replace(' ', '_')}, inplace=True)
data.starttime = pd.to_datetime(data.starttime)
data.stoptime = pd.to_datetime(data.stoptime)

# Rohdaten sichten
data

In [ ]:
# Sanity checks der Rohdaten

print('Tripduration')
print(f'Min: {np.min(data.tripduration)} s, Max: {np.max(data.tripduration)/3600:.0f} h\n')

print('Starttime')
print(f'Min: {np.min(data.starttime)}, Max: {np.max(data.starttime)}\n')

print('Stoptime')
print(f'Min: {np.min(data.stoptime)}, Max: {np.max(data.stoptime)}\n')

print('Start station id')
print(f'Min: {np.min(data.start_station_id)}, Max: {np.max(data.start_station_id)}, #: {len(np.unique(data.start_station_id))}\n')

# print('Start station name')
# print(f'#: {len(np.unique(data.start_station_name))}\n')
      
print('Start station latitude')
print(f'Min: {np.min(data.start_station_latitude)}, Max: {np.max(data.start_station_latitude)}, #: {len(np.unique(data.start_station_latitude))}\n')

print('Start station longtude')
print(f'Min: {np.min(data.start_station_longitude)}, Max: {np.max(data.start_station_longitude)}, #: {len(np.unique(data.start_station_longitude))}\n')

print('End station id')
print(f'Min: {np.min(data.end_station_id)}, Max: {np.max(data.end_station_id)}, #: {len(np.unique(data.end_station_id))}\n')

# print('End station name')
# print(f'#: {len(np.unique(data.end_station_name))}\n')

print('End station latitude')
print(f'Min: {np.min(data.end_station_latitude)}, Max: {np.max(data.end_station_latitude)}, #: {len(np.unique(data.end_station_latitude))}\n')

print('End station longitude')
print(f'Min: {np.min(data.end_station_longitude)}, Max: {np.max(data.end_station_longitude)}, #: {len(np.unique(data.end_station_longitude))}\n')

print('Bikeid')
print(f'Min: {np.min(data.bikeid)}, Max: {np.max(data.bikeid)}, #: {len(np.unique(data.bikeid))}\n')

print('Usertype')
print(f'Types: {np.unique(data.usertype)}, #: {np.unique(data.usertype, return_counts=True)[1]}\n')

print('Birth year')
print(f'Min: {np.min(data.birth_year)}, Max: {np.max(data.birth_year)}\n')

print('Gender')
print(f'Types: {np.unique(data.gender)}, #: {np.unique(data.gender, return_counts=True)[1]}\n')

In [ ]:
# Daten säubern

# Sortiere Fahrten aus, die länger als 2 Stunden dauerten
len_before = len(data)
data.drop(data[data.tripduration >= max_trip_duration].index, inplace=True)
len_after = len(data)
print(f'Dropped {len_before-len_after} lines due to excessive tripduration')

# Macht es Sinn, Geburtsjahre vor 1918 auszusortieren?

### Erstes Fazit

- Datensatz sehr groß, wir beschränken uns deswegen auf die Sommermonate
- Datensatz ist 'imbalanced', 19 000 customer gegen 156 000 subscriber
- Datenqualität ist okay, aber ein paar sehr alte Kunden, lange Fahrten und unhomogene Koordinatenangaben
- Wir sollten schon vor der weiteren Analyse einen Teil der Daten beiseitelegen, um eine wirklich unabhängige Evaluation unseres Modells zu erreichen
- Wir sollten schon bei der visuellen Analyse des Datensatzes die Aufgabenstellung im Auge behalten

## Features für Customer-Subscriber-Modell

In [ ]:
data['start_hour_of_day']= [t.hour for t in data.starttime]

data['start_day_of_week'] = [t.dayofweek for t in data.starttime]

data['start_day_of_year'] = [t.dayofyear for t in data.starttime]

data['duration_seconds'] = data.tripduration

data['crow_distance_meters'] = crow_distance(data.start_station_latitude, data.start_station_longitude, data.end_station_latitude, data.end_station_longitude)

data['crow_velocity_meters_per_second'] = data.crow_distance_meters / data.duration_seconds

data['age'] = 2018 - data.birth_year

data['gender_male'] = [1 if g==1 else 0 for g in data.gender]

data['gender_female'] = [1 if g==2 else 0 for g in data.gender]

data['gender_unknown'] = [1 if g==0 else 0 for g in data.gender]

data['customer'] = [1 if u=='Customer' else 0 for u in data.usertype]

data['subscriber'] = [1 if u=='Subscriber' else 0 for u in data.usertype]

data['set'] = [np.random.choice(['train', 'dev', 'test'], p=[0.8, 0.1, 0.1]) for r in range(len(data))]

data['train_set'] = [True if s=='train' else False for s in data.set]

data['dev_set'] = [True if s=='dev' else False for s in data.set]

data['test_set'] = [True if s=='test' else False for s in data.set]

data

## Visualisierungen

In [ ]:
# histogramme, sortiert nach customer und Subscriber
plot_specs = [('start_hour_of_day', range(25)),
              ('start_day_of_week', range(7)),
              ('start_day_of_year', np.linspace(0, 366, num=20)),
              ('duration_seconds', np.linspace(0, 3600, num=20)),
              ('crow_distance_meters', np.linspace(0, 6000, num=20)),
              ('crow_velocity_meters_per_second', np.linspace(0, 6, num=20)),
              ('age', np.linspace(0, 80, num=20)),
              ('gender_male', range(3)),
              ('gender_female', range(3)),
              ('gender_unknown', range(3))]

data_customer = data[~data.test_set & data.customer==1]
data_subscriber = data[~data.test_set & data.subscriber==1]

for column, bins in plot_specs:
    plt.hist([data_customer[column], data_subscriber[column]], bins, stacked=True, rwidth=.95, color=['tomato', 'silver'])
    plt.title(f'{column} histogram')
    plt.show()
    plt.close()

In [ ]:
# Wegstrecken, ebenfalls nach customer und subscriber 

for idx, row in data_customer.iterrows():
    plt.plot([row.start_station_longitude, row.end_station_longitude], [row.start_station_latitude, row.end_station_latitude], alpha=0.01, color='tomato')
plt.title('customer frequent routes')
plt.show()
plt.close()

data_subscriber_for_plot = data_subscriber.drop(np.random.choice(data_subscriber.index, int(len(data_subscriber)*0.9), replace=False))
for idx, row in data_subscriber.iterrows():
    plt.plot([row.start_station_longitude, row.end_station_longitude], [row.start_station_latitude, row.end_station_latitude], alpha=0.01, color='silver')
plt.title('subscriber frequent routes')
plt.show()
plt.close()

## Modelle

In [ ]:
# Inputs definieren
feature_columns = ['start_hour_of_day',
                   'start_day_of_week',
                   'start_day_of_year',
                   'duration_seconds',
                   'crow_distance_meters',
                   'crow_velocity_meters_per_second',
                   'age',
                   'gender_male',
                   'gender_female',
                   'gender_unknown']

target_column = ['customer']

# Training und validation data
x_train = data.loc[data.train_set, feature_columns]
x_dev = data.loc[data.dev_set, feature_columns]
y_train = data.loc[data.train_set, target_column]
y_dev = data.loc[data.dev_set, target_column]

In [ ]:
# Erstes simples Modell: klassifiziere immer als Subscriber
def S(x):
    return [0] * len(x)

y_dev_hat = simple_model(x_dev)

# Confusion matrix
disp = ConfusionMatrixDisplay.from_predictions(y_dev, y_dev_hat)
plt.title('confusion matrix for linear classifier')
plt.show()
plt.close()


In [ ]:
# Lineare Klassifikation
linear_model = SGDClassifier(max_iter=1000, tol=1e-13)
linear_model.fit(x_train, y_train)
y_dev_hat = linear_model.predict(x_dev)

# Confusion matrix
disp = ConfusionMatrixDisplay.from_predictions(y_dev, y_dev_hat)
plt.title('confusion matrix for linear classifier')
plt.show()
plt.close()

In [ ]:
# Random forest für Klassifikation
random_forest = RandomForestClassifier()
random_forest.fit(features_train, )
y_dev_hat = RF.predict(x_dev)

# Confusion matrix
disp = ConfusionMatrixDisplay.from_predictions(y_dev, y_dev_hat)
plt.title('confusion matrix for linear classifier')
plt.show()
plt.close()

In [ ]:
# Random forest Modell erklären
explainer = shap.TreeExplainer(random_forest, x_dev)
shap_values = explainer(x_dev)
shap.plots.beeswarm(shap_values)

In [ ]:
# Welche features sind sinnvoll?



## Use cases für Modelle

- Fahrräder bereitstellen
- Kunden elastizität, vielleicht Pauschale erhöhen?

##  Kooperation mit Versicherung

- ...

## Was ich noch gemacht hätte, wenn ich Zeit gehabt hätte

- Datensatz als Graph betrachten, zwischen welchen Stationen fahren Customer und Subscriber gerne?
- ...